<a href="https://colab.research.google.com/github/Reckonchamp12/AI-Bitcoin-Project/blob/main/Bitcoin_Colab_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive

In [2]:
# Mount Google Drive to save models
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load Bitcoin data
DATA_URL = "https://www.cryptodatadownload.com/cdd/Bitstamp_BTCUSD_d.csv"
df = pd.read_csv(DATA_URL, skiprows=1)
df = df[::-1]  # Reverse order
df = df[['date', 'close']]
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

In [4]:
# Normalize data
scaler = MinMaxScaler()
data = scaler.fit_transform(df['close'].values.reshape(-1, 1))

In [5]:
# Create dataset for LSTM
def create_sequences(data, seq_length=30):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i+seq_length])
        labels.append(data[i+seq_length])
    return np.array(sequences), np.array(labels)

seq_length = 30
X, y = create_sequences(data, seq_length)
X = torch.FloatTensor(X)
y = torch.FloatTensor(y)


In [6]:
class BitcoinLSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, num_layers=2, output_size=1):
        super(BitcoinLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out


In [7]:
# Initialize model, loss function, optimizer
model = BitcoinLSTM()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
# Train Model
epochs = 10
for epoch in range(epochs):
    optimizer.zero_grad()
    output = model(X)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.6f}")


Epoch 1/10, Loss: 0.138277
Epoch 2/10, Loss: 0.126017
Epoch 3/10, Loss: 0.114675
Epoch 4/10, Loss: 0.104136
Epoch 5/10, Loss: 0.094324
Epoch 6/10, Loss: 0.085182
Epoch 7/10, Loss: 0.076675
Epoch 8/10, Loss: 0.068800
Epoch 9/10, Loss: 0.061597
Epoch 10/10, Loss: 0.055163


In [9]:

# Save trained model to Google Drive
torch.save(model.state_dict(), '/content/drive/MyDrive/bitcoin_lstm.pth')
print("Model saved to Google Drive!")


Model saved to Google Drive!
